Для корректной работы ноутбука необходимо распаковать файл data.zip в ту же директорию

In [1]:
# !pip install gensim
# !pip install razdel
# !pip install pymorphy2
# !pip install pyenchant
# !pip install python-Levenshtein

In [2]:
# import nltk
# nltk.download('stopwords')

In [3]:
import numpy as np
import pandas as pd
import re
import pymorphy2
import itertools

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score
from sklearn.metrics import classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from razdel import tokenize
from gensim.models import LdaModel
from gensim.test.utils import datapath
from gensim.corpora.dictionary import Dictionary

import warnings
warnings.simplefilter("ignore")
%matplotlib inline

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или иной способ оказался эффективнее остальных

___

In [4]:
NUMBER_OF_TOPICS = 25

In [5]:
def has_cyr(text: str) -> bool:
    """
    Проверяем, кириллическое ли слово
    """
    return bool(re.search('[а-яА-Я]', text))

In [6]:
def clean_text(text):
    """
    очистка текста

    на выходе относительно очищенный текст

    """
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', text)
    text = re.sub(r"[n]", ' ', text)

    text = re.sub("[0-9]|[.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

In [7]:
def lemmatization(text: str, stopword: list):
    """
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    """

    cache = {}
    morph = pymorphy2.MorphAnalyzer()
    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-':  # [2]
            w = w[1:]
        if len(w) > 1:  # [3]
            if w in cache:  # [4]
                words_lem.append(cache[w])
            else:  # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [i for i in words_lem if i not in stopword]  # [6]

    return words_lem_without_stopwords

In [8]:
def str_to_list(some_str: str) -> list:
    """
    Функция переводит из строкового типа в лист, отрезая лишние пробелы и кавычки
    Потребовалась, так как read_csv из листа сделал строку.
    """
    some_list = some_str.strip('[]').split(',')
    return [word.strip().strip("''") for word in some_list]

In [9]:
def get_lda_vector(text, lda_model):
    lda_tuple = lda_model[common_dictionary.doc2bow(text)]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(NUMBER_OF_TOPICS):
        output_vector.append(not_null_topics[i]) if i in not_null_topics else output_vector.append(0)
    return np.array(output_vector)

In [10]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [11]:
def get_user_embedding(user_articles_list, doc_dict, idf_dict, func='mean'):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if func == 'mean':
        user_vector = np.mean(user_vector, axis=0)
    elif func == 'median':
        user_vector = np.median(user_vector, axis=0)
    elif func == 'max':
        user_vector = np.max(user_vector, axis=0)
    elif func == 'idf-mean':
        # Скалярно умножим пользовательский вектор на вектор весов idf
        idf_vector = np.array([idf_dict[doc_id] for doc_id in user_articles_list])
        user_vector = user_vector.T.dot(idf_vector)
    else:
        raise ValueError("func must be from [mean, median, max, idf-mean]")
    return user_vector

In [12]:
def embed_users(users, topic_matrix, target, func='mean', verbose=True):
    """
    Функция принимает на вход матрицу топиков, целевую переменную и функцию, которая
    применяется на вектор пользователей (mean, median, max, idf-mean).
    На выходе вектора предсказаний, реальных значений, thr, pr, rec, f1 и индекс лучшего значения
    """
    doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]].values))
    # Создадим аналогичный предыдущему словарь, содержащий пары значений статья: ее средний idf
    idf_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix['idf_mean'].values))
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, idf_dict, func=func), 1)])

    user_embeddings.columns = [f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]

    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] + [f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]]
    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]],
                                                        X['churn'], random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)

    y_preds = logreg.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, y_preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # Так как precision и recall могут быть одновременно нулями - получим nan в массиве.
    # Приравняем их к 0
    np.nan_to_num(fscore, copy=False)

    # locate the index of the largest f score
    ix = np.argmax(fscore)
    if verbose:
        print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], fscore[ix],
                                                                                precision[ix], recall[ix]))

    return y_preds, y_test, thresholds, precision, recall, fscore, ix

In [13]:
def pipe_transform(text: str, stopword: list):
    """
    Костыль для последовательной трансформации текста
    """
    return lemmatization(clean_text(text), stopword)

In [14]:
def idf_mean(article, com_dict, corpus_len):
    """
    Функция подсчета среднего idf для токенизированного документа. На входе список слов документа,
    словарь в формате corpora.dictionary и общее количество документов
    """
    result = []
    for word in set(article):
        result.append(np.log(corpus_len / com_dict.dfs[com_dict.token2id[word]]))
    return np.mean(result)

___

In [15]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")
target = pd.read_csv("users_churn.csv")
stopwords_ru = stopwords.words('russian')

In [16]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

In [17]:
# Сделаем небольшую модификацию кода, будем добавлять только русские слова в стоп-список
add_words_ru = [word for word in additional_stopwords if has_cyr(word)]
stopwords_ru += add_words_ru

In [18]:
# Так как это добро на моем ноутбуке считалось часа 2, записал в csv
# news['title'] = news['title'].apply(lambda x: pipe_transform(x, stopwords_ru))
# news.to_csv('./news.csv')

In [19]:
# Дропаем получившийся после сохранения ненужный стобец с индексами
mod_news = pd.read_csv('news.csv').drop('Unnamed: 0', axis=1)

# Приводим нужный нам столбец в нормальный вид (из строки в лист)
mod_news['title'] = mod_news['title'].apply(lambda x: str_to_list(x))

In [20]:
common_dictionary = Dictionary(mod_news['title'].values)
common_corpus = [common_dictionary.doc2bow(text) for text in mod_news['title'].values]

In [21]:
mod_news.head()

,doc_id,title
0,6,"[заместитель, председатель, правительство, рф,..."
1,4896,"[матч, финал, кубок, россия, футбол, приостано..."
2,4897,"[форвард, авангард, томаш, заборский, прокомме..."
3,4898,"[главный, тренер, кубань, юрий, красножанин, п..."
4,4899,"[решение, попечительский, совет, владивостокск..."


In [22]:
# Посчитаем вектор средних idf весов для каждого документа
idf_vector = [idf_mean(mod_news['title'].iloc[i], common_dictionary, mod_news.shape[0]) 
              for i in range(mod_news.shape[0])]

In [23]:
lda_file = datapath("model.lda")
# Обучили и сохранили. Потом просто вытащим из файла
# lda = LdaModel(common_corpus, num_topics=NUMBER_OF_TOPICS, id2word=common_dictionary)
# lda.save(lda_file)

In [24]:
lda = LdaModel.load(lda_file)

In [25]:
topic_matrix = pd.DataFrame([get_lda_vector(text, lda) for text in mod_news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]
topic_matrix['doc_id'] = mod_news['doc_id'].values
topic_matrix = topic_matrix[['doc_id'] + [f'topic_{i}' for i in range(NUMBER_OF_TOPICS)]]
topic_matrix['idf_mean'] = idf_vector

In [26]:
topic_matrix.head()

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,idf_mean
0,6,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.081637,0.0,0.108471,0.000000,4.082629
1,4896,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.974720,0.000000,4.170311
2,4897,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.703102,0.041559,4.194190
3,4898,0.101621,0.0,0.0,0.0,0.000000,0.0,0.0,0.016286,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.479164,0.000000,3.824732
4,4899,0.183356,0.0,0.0,0.0,0.044792,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.419067,0.328033,0.0,0.000000,0.000000,4.202383


In [27]:
emd_mode = ['mean', 'median', 'max', 'idf-mean']
col_names = ['threshhold', 'roc-auc', 'f1-score', 'precision', 'recall']
result = []
for mode in emd_mode:
    y_preds, y_test, thresholds, precision, recall, fscore, ix = embed_users(users, topic_matrix, 
                                                                             target, func=mode, 
                                                                             verbose=False)
    result.append([thresholds[ix], roc_auc_score(y_test, y_preds), 
                   fscore[ix], precision[ix], recall[ix]])
result_df = pd.DataFrame(result, index=emd_mode, columns=col_names)

In [28]:
result_df

,threshhold,roc-auc,f1-score,precision,recall
mean,0.254812,0.944622,0.669159,0.617241,0.730612
median,0.264692,0.971838,0.781925,0.753788,0.812245
max,0.422737,0.978555,0.806867,0.850679,0.767347
idf-mean,0.437304,0.973403,0.790419,0.773438,0.808163


Самый слабый результат получили при методе эмбединга mean. 
Остальные результаты могут быть использованы при разных требованиях заказчика:
- Медиану для приоритета recall, то есть когда важно не пропустить целевой класс, пусть и с большим количеством ложно-положительных
- Максимальное значение в обратном случае, когда важнее недопустить ложно-положительных
- Взвешивание по idf в случае баланса precision и recall